In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import anndata as ad
import seaborn as sns
import muon as mu
from muon import atac as ac
import random
import os
import openpyxl
import scipy
from scipy.stats import median_abs_deviation

In [3]:
adata = sc.read_10x_mtx('/Users/alexandra/Desktop/Data/Single-Cell/hg19_10xCloud_aligned_data/filtered_feature_bc_matrix', gex_only = False)

gex_rows = list(map(lambda x: x == 'Gene Expression', adata.var['feature_types']))
atac_rows = list(map(lambda x: x == 'Peaks', adata.var['feature_types']))

adata_gem = adata[:, gex_rows].copy()
adata_atac = adata[:, atac_rows].copy()

adata_gem.var_names_make_unique()
adata_atac.var_names_make_unique()

In [ ]:
# mitochondrial genes
adata_gem.var["mt"] = adata_gem.var_names.str.startswith("MT-")
# ribosomal genes
adata_gem.var["ribo"] = adata_gem.var_names.str.startswith(("RPS", "RPL"))
# hemoglobin genes.
adata_gem.var["hb"] = adata_gem.var_names.str.contains("^HB[^(P)]")

In [ ]:
# calculate qc metrics
sc.pp.calculate_qc_metrics(
    adata_gem, qc_vars=["mt", "ribo", "hb"], inplace=True, percent_top=[20], log1p=True
)
adata_gem

In [ ]:
p1 = sc.pl.violin(adata_gem, ["pct_counts_mt"])
p2 = sc.pl.scatter(adata_gem, x="total_counts", y="n_genes_by_counts")
p3 = sns.displot(adata_gem.obs["total_counts"], bins=50, kde=True)

In [ ]:
# filter cells by MADs
def is_outlier(adata, metric: str, nmads: int):
    M = adata.obs[metric]
    outlier = (M < np.median(M) - nmads * median_abs_deviation(M)) | (np.median(M) + nmads * median_abs_deviation(M) < M )
    return outlier 

In [ ]:
# filter cells by MADs
adata_gem.obs["outlier"] = (
    is_outlier(adata_gem, "total_counts", 5)
  | is_outlier(adata_gem, "n_genes_by_counts", 5)
)

adata_gem.obs.outlier.value_counts()

In [ ]:
adata_gem.obs["mt_outlier"] = adata_gem.obs["pct_counts_mt"] > 20
adata_gem.obs.mt_outlier.value_counts()

In [ ]:
adata_gem = adata_gem[~adata_gem.obs["outlier"] & ~adata_gem.obs.mt_outlier].copy()
adata_gem

In [ ]:
p1 = sc.pl.violin(adata_gem, ["pct_counts_mt"])
p2 = sc.pl.scatter(adata_gem, x="total_counts", y="n_genes_by_counts")
p3 = sns.displot(adata_gem.obs["total_counts"], bins=50, kde=True)

In [ ]:
# Optional: keep total validation gene
valid_csv = pd.read_excel("/Users/alexandra/Desktop/Data/CRISPRiFlowFISH/41588_2019_538_MOESM3_ESM.xlsx",  sheet_name="Supplementary Table 6a", skiprows=1)
valid_gene_list = set(valid_csv["Gene"])
len(valid_gene_list)

In [ ]:
# Optional: keep only CRIPRIi-FlowFISH validation gene
valid_csv = pd.read_excel("/Users/alexandra/Desktop/Data/CRISPRiFlowFISH/41588_2019_538_MOESM3_ESM.xlsx",  sheet_name="Supplementary Table 3a", skiprows=1)
valid_gene_list = set(valid_csv["Gene"])
len(valid_gene_list)

In [ ]:
# Optional: add validation genes back
valid_gene_data = adata_gem[:,pd.Index(valid_gene_list)].copy()

In [ ]:
# fitler genes
sc.pp.filter_genes(adata_gem, min_cells=20)
adata_gem = adata_gem[:,~adata_gem.var["mt"]& ~adata_gem.var["ribo"] & ~adata_gem.var["hb"]].copy()
adata_gem

In [ ]:
# Normalization
sc.pp.normalize_total(adata_gem, target_sum=1e6)
sc.pp.log1p(adata_gem)

In [ ]:
adata_gem

In [ ]:
# Select highly variable genes
adata_gem_hvg = adata_gem.copy()
sc.pp.highly_variable_genes(adata_gem_hvg, n_top_genes=3000, inplace=True)

adata_gem_hvg = adata_gem_hvg[:, adata_gem_hvg.var.highly_variable].copy()

In [ ]:
# UMAP
sc.pp.neighbors(adata_gem_hvg, n_neighbors=15, n_pcs=50)
sc.tl.umap(adata_gem_hvg, alpha = 1, gamma = 0.1)

In [ ]:
# clustering
sc.tl.leiden(adata_gem_hvg)

In [ ]:
sc.pl.umap(adata_gem_hvg, color=["leiden"], legend_loc="on data", ncols=3)

In [ ]:
# Optional: add validation genes back
adata_gem = ad.concat([adata_gem, valid_gene_data], axis=1, join="outer")

In [ ]:
#ATAC-seq peak length
peak_list = list(adata_atac.var_names)

length = []
for peak in peak_list:
    _,coordnats = peak.split(":")
    start,end = map(int,coordnats.split("-"))
    length.append(end-start)
    
average_length = np.mean(length)
print("Average length for ATAC-seq is:", average_length)

Average length for ATAC-seq is: 847.644787143379


In [ ]:
# calcaulte genereal qc metrics for ATAC-seq
sc.pp.calculate_qc_metrics(adata_atac,percent_top = None, log1p = False, inplace = True)

adata_atac.obs.rename(columns = {"n_genes_by_counts":"n_fragment_by_counts","total_counts":"total_fragment_counts"}, inplace = True)

adata_atac.obs["log_total_fragment_counts"] = np.log1p(adata_atac.obs["total_fragment_counts"])

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [ ]:
# calculate nucleosome signal
mu.atac.tl.locate_file(adata_atac, file = "/Volumes/G-DRIVE mobile USB-C/Single-cell_data/K562/10x/ISSAACC-seq_generated/hg19_10xCloud_aligned_data/atac_fragments.tsv.gz", key = "fragments")
ac.tl.nucleosome_signal(adata_atac, n=10e3 * adata_atac.n_obs)

In [ ]:
# calculate tss enrichment
df_annotation = pd.read_csv("/Volumes/G-DRIVE mobile USB-C/Single-cell_data/K562/10x/ISSAACC-seq_generated/hg19_10xCloud_aligned_data/atac_peak_annotation.tsv", sep='\t')
df_annotation = df_annotation[['chrom', 'start', 'end']]
df_annotation.columns = ['Chromosome', 'Start', 'End']

tss = ac.tl.tss_enrichment(adata_atac,  n_tss=3000, random_state=666, features = df_annotation)

Fetching Regions...: 100%|██████████| 3000/3000 [00:49<00:00, 60.00it/s]
/opt/anaconda3/envs/CLR/lib/python3.11/site-packages/anndata/_core/anndata.py:401: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
